# Neonatal brain measure estimation from dHCP data
As dHCP is under data protection, no data is included in this repository. The code is provided as a reference for the analysis of the dHCP data. To reproduce the analyses, you need to apply for access to the dHCP dataset [here](https://nda.nih.gov/edit_collection.html?id=3955), and parcellate the data with the updated version of the [MCRIBS atlas](https://github.com/DevelopmentalImagingMCRI/MCRIBS). Change the directory `mcribs_data_dir` accordingly. 

Data can be requested from the authors among reasonable request. I received parcellated data of dHCP release 3 from my collaborators C. Adamson, G. Ball, R. Beare, and J.A.S. de Almeida.

This whole notebook should be run for cortical thickness (CT) or surface area (SA). Both measures were used in the original publication. Adjust the variable `brain_measure` to `CT` or `SA` accordingly. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import subprocess
from functools import reduce
from abagen import fetch_desikan_killiany

import sys
sys.path.append('code')
from preprocessing import dhcp_assign_scan_label, dhcp_format_interview_date, dhcp_read_meta, dhcp_strip_id 
from utils import reorder_vars


dhcp_data_dir = 'data/dHCP'
mcribs_data_dir = 'data/dHCP/rel3/dhcp-MCRIBS'  # where MCRIBS parcellated data are saved
dhcp_data_dir_rel = 'data/dHCP/rel3'  # where the dHCP data are saved

dhcp_freesurfer_outputs = join(dhcp_data_dir, 'freesurfer')  # where all freesurfer outputs are stored
dhcp_out = join(dhcp_data_dir, 'derivatives')  # where the adapted data will be stored
os.makedirs(dhcp_out, exist_ok=True)


# adjust brain measurement for which the code should be run
brain_measure = 'CT'  # CT or SA

# Load MRI related data
The dHCP dataset has been published in several releases. Data from release 2 and 3 will be used here. MRI data were parcellated using the MCRIBS atlas, which is a neonatal brain atlas [developed](https://doi.org/10.1038/s41598-020-61326-2) and [updated](https://doi.org/10.1007/s12021-024-09656-8) by Chris Adamson and colleagues. The MCRIBS atlas is available on [GitHub](https://github.com/DevelopmentalImagingMCRI/MCRIBS). The output generated by MCRIBS is similar to FreeSurfer's ?.aparc.annot files and corresponds to the parcellation of the adult Desikan-Killiany atlas. 

In the updated version of the manuscript, dHCP release 3 with the updated MCRIBS parcellation is used. Therefore, data from release 2 and 3 will be combined.

In [3]:
# define aseg columns to be extracted
aseg_cols = ['SubCortGrayVol', 'CerebralWhiteMatterVol', 'CortexVol']

## Load MRI data
### Release 2

In [4]:
# aseg
rel2_aseg = pd.read_csv(join(mcribs_data_dir, 'dHCP2_aseg.csv'), index_col=0, sep=',')
rel2_aseg.index.name = 'participant'  
rel2_aseg = rel2_aseg[aseg_cols] 

# brain measure
if brain_measure == 'CT':
    rel2_mri_lh = pd.read_csv(join(mcribs_data_dir, 'dHCP2_lh.dk.ThickAvg.csv'), index_col=0, sep=',')
    rel2_mri_rh = pd.read_csv(join(mcribs_data_dir, 'dHCP2_rh.dk.ThickAvg.csv'), index_col=0, sep=',')
    rel2_mri = rel2_mri_lh.merge(rel2_mri_rh, on='ThickAvg')
elif brain_measure == 'SA':
    rel2_mri_lh = pd.read_csv(join(mcribs_data_dir, 'dHCP2_lh.dk.SurfArea.csv'), index_col=0, sep=',')
    rel2_mri_rh = pd.read_csv(join(mcribs_data_dir, 'dHCP2_rh.dk.SurfArea.csv'), index_col=0, sep=',')
    rel2_mri = rel2_mri_lh.merge(rel2_mri_rh, on='SurfArea')
else: 
    raise ValueError("brain_measure must be either 'CT' or 'SA'")

# drop medial wall columns and rename index
rel2_mri.drop(columns=['Medial_Wall_x', 'Medial_Wall_y'], inplace=True)  # remove participant column as it is not needed
rel2_mri.index.name = 'participant'  

# merge brain_measure and aseg data
rel2 = rel2_mri.merge(rel2_aseg, on='participant', how='outer')

# separate session and participant id
rel2 = dhcp_strip_id(rel2)
rel2['release'] = 2
print(f"Number of participants in release 2: {len(rel2)}")

Number of participants in release 2: 542


### Release 3

In [5]:
# aseg
rel3_aseg = pd.read_csv(join(mcribs_data_dir, 'dHCP3_aseg.csv'), index_col=0, sep=',')
rel3_aseg.index.name = 'participant'  
rel3_aseg = rel3_aseg[aseg_cols]

# brain measure
if brain_measure == 'CT':
    rel3_mri_lh = pd.read_csv(join(mcribs_data_dir, 'dHCP3_lh.dk.ThickAvg.csv'), index_col=0, sep=',')
    rel3_mri_rh = pd.read_csv(join(mcribs_data_dir, 'dHCP3_rh.dk.ThickAvg.csv'), index_col=0, sep=',')
    rel3_mri = rel3_mri_lh.merge(rel3_mri_rh, on='ThickAvg')
elif brain_measure == 'SA':
    rel3_mri_lh = pd.read_csv(join(mcribs_data_dir, 'dHCP3_lh.dk.SurfArea.csv'), index_col=0, sep=',')
    rel3_mri_rh = pd.read_csv(join(mcribs_data_dir, 'dHCP3_rh.dk.SurfArea.csv'), index_col=0, sep=',')
    rel3_mri = rel3_mri_lh.merge(rel3_mri_rh, on='SurfArea')
else: 
    raise ValueError("brain_measure must be either 'CT' or 'SA'")

# drop medial wall columns and rename index
rel3_mri.drop(columns=['Medial_Wall_x', 'Medial_Wall_y'], inplace=True)  # remove participant column as it is not needed
rel3_mri.index.name = 'participant'  

# merge brain_measure and aseg data
rel3 = rel3_mri.merge(rel3_aseg, on='participant', how='outer')

# separate session and participant id
rel3 = dhcp_strip_id(rel3)
rel3['release'] = 3
print(f"Number of participants in release 3: {len(rel3.index)}")

Number of participants in release 3: 310


In [6]:
# combine data from release 2 and 3
dhcp_mri = pd.concat([rel2, rel3], axis=0)
dhcp_mri.sort_values(by=['Subject_ID', 'Session_ID'], inplace=True)
print(f"Combined dHCP data shape: {dhcp_mri.shape}")


# get region names of the Desikan-Killiany atlas as now used in the dHCP data
dhcp_idps_idx_wo_nan = dhcp_mri.filter(regex=f'ctx-').columns
print(f"Number of IDPs in DKT atlas: {len(dhcp_idps_idx_wo_nan)}")

# mean CT
if brain_measure == 'CT':
    dhcp_mri['meanCT2'] = dhcp_mri[dhcp_idps_idx_wo_nan].mean(axis=1)

Combined dHCP data shape: (852, 74)
Number of IDPs in DKT atlas: 68


<!-- ## Match DKT to DK atlas
Some regions are missing in the DKT atlas that are present in the DK atlas (bankssts, frontalpole, temporalpole). -->

## Load DK atlas

In [7]:
# access DK atlas and save idp_labels
atlas = fetch_desikan_killiany(surface=True)
atlas = pd.read_csv(atlas['info'])
atlas = atlas[(atlas['structure'] == 'cortex') & (atlas['hemisphere'] == 'L')]
atlas_labels_l = ['L_'+label for label in atlas['label']]
atlas_labels_r = ['R_'+label for label in atlas['label']]

desikan_idps = atlas_labels_l + atlas_labels_r  # save correct labels to rename the columns later

In [8]:
# reorder columns of dhcp_mri
dk_labels_raw = atlas['label'].tolist()
dk_labels_dhcp = [f'ctx-{hemisphere}-{label}' for hemisphere in ['lh', 'rh'] for label in dk_labels_raw]
dhcp_mri = reorder_vars(['Subject_ID', 'Session_ID', 'release'], dhcp_mri, dk_labels_dhcp)

# sorting dhcp_idps_idx
dhcp_idps_idx = dhcp_mri.filter(regex=f'ctx-').columns

# Meta information

In [9]:
# load meta information
meta = dhcp_read_meta(join(dhcp_data_dir_rel, 'nnsi01.txt'))
meta = meta[['src_subject_id', 'interview_date', 'sex', 'nscan_ga_at_birth_weeks', 'nscan_ga_at_scan_weeks', 
            'radiology_score', 'scan_validation']]
meta['scan_validation'] = ['ses-' + str(ses) for ses in meta['scan_validation']] 

# rename columns and adjust data types
meta.rename(columns={'src_subject_id': 'Subject_ID', 
                    'scan_validation': 'Session_ID',
                    'nscan_ga_at_birth_weeks': 'gestational_age', 
                    'nscan_ga_at_scan_weeks': 'scan_age'}, inplace=True)

meta['scan_age'] = meta['scan_age'].astype(float)
meta['gestational_age'] = meta['gestational_age'].astype(float)

# format interview date to be consistent across subjects
meta['interview_date_formatted'] = meta['interview_date'].apply(dhcp_format_interview_date)
meta.drop(columns=['interview_date'], inplace=True)

# assign session and scan labels
meta['session_num'] = meta.groupby('Subject_ID').cumcount() + 1
meta['scan'] = meta.apply(dhcp_assign_scan_label, axis=1)

## Merge meta with MRI data  
Although some subjects have multiple scans (i.e., term-equivalent and fetal), we will only keep the term-equivalent scan for each subject. This is because the normative model will be trained on term-born subjects only, for whom only term-equivalent scans are available.

In [10]:
dhcp = dhcp_mri.merge(meta, on=['Subject_ID', 'Session_ID'], how='inner')

In [11]:
# only keep term-equivalent scans
dhcp = dhcp[dhcp['scan'] == 'term_equivalent']
print(f"Final dHCP data shape: {dhcp.shape}")

# if subject has multiple scans at term-equivalent age, keep the last one
termeq_df = dhcp[dhcp['scan'] == 'term_equivalent']
termeq_df = termeq_df.sort_values(by=['Subject_ID', 'Session_ID']).groupby('Subject_ID').last().reset_index()

fetal_df = dhcp[dhcp['scan'] == 'fetal']
dhcp = pd.concat([termeq_df, fetal_df], ignore_index=True)

print(f"Final dHCP data shape after keeping only last term-equivalent scan: {dhcp.shape}")

Final dHCP data shape: (701, 82)
Final dHCP data shape after keeping only last term-equivalent scan: (695, 82)


# Load data about cognition and birth

In [12]:
dhcp_tab_data = {
    join(dhcp_data_dir_rel, 'bsid_iii01.txt'):  ['bsid_cog_composite'], 
    # join(dhcp_data_dir_rel, 'cpenr01.txt'):     ['mother_education1', 'mother_occupation', 'father_education1', 'father_occupation'], 
    join(dhcp_data_dir_rel, 'lpb01.txt'):       ['baby_birth_weight'], 
}

# read data
dhcp_tab = []
for k in dhcp_tab_data.keys():
    tab = pd.read_csv(k, header=0, skiprows=[1], delimiter="\t", na_values=['-998', '-999', ''], 
                        low_memory=False)
    tab = tab[["src_subject_id"] + dhcp_tab_data[k]]     
    dhcp_tab.append(tab)
    
# combine data
dhcp_tab = reduce(lambda left, right: pd.merge(left, right, on=["src_subject_id"], how='left'), dhcp_tab)

# rename columns
dhcp_tab.rename(columns={'src_subject_id': 'Subject_ID',
                        'baby_birth_weight': 'BW'}, inplace=True)

# transform BW into grams
dhcp_tab['BW'] = dhcp_tab['BW'] * 1000  # convert kg to grams

## Adjust and rename variables

In [13]:
# merge 
dhcp = dhcp.merge(dhcp_tab, on=['Subject_ID'], how='left') 


# rename variables
dhcp = dhcp.rename(columns=dict(
    SubCortGrayVol="sGMV",
    CerebralWhiteMatterVol="WMV",
    CortexVol="GMV"
))  

# reorder variables to match the Desikan-Killiany atlas
dhcp = dhcp.rename(columns=dict(zip(dhcp_idps_idx, desikan_idps)))

In [14]:
# add preterm variable
dhcp['dx'] = np.where(dhcp['gestational_age'] < 37, 'preterm', 'CN')

# Average hemispheres

In [15]:
regions = [r[2:] for r in desikan_idps[:34]]

# Combine L_ and R_ values
for region in regions:
    dhcp[f'{brain_measure}_{region}'] = dhcp[[f'L_{region}', f'R_{region}']].mean(axis=1)
    # drop L_ and R_ columns
    dhcp = dhcp.drop(columns=[f'L_{region}', f'R_{region}'])

In [16]:
# create region list bilateral
desikan_idps_bilateral = [f'{brain_measure}_{region}' for region in regions]
ctv_columns = ['GMV', 'WMV', 'sGMV']

# Adapt df for BrainChart framework
BrainChart needs a certain format of the data. We will adapt the data accordingly. More information can be found [here](https://brainchart.shinyapps.io/brainchart/).

In [17]:
# adapt for brainchart
dhcp = dhcp.rename(columns=dict(
    Subject_ID="participant",
    gestational_age="GA",
    baby_birth_weight="BW",
)) 

# sex
dhcp['sex'] = dhcp['sex'].replace({'F': 'Female', 'M': 'Male'})

# age
dhcp['age_days'] = dhcp['scan_age'] * 7  # because scan_age is in weeks
dhcp['Age'] = (dhcp['age_days']-280) / 365.245  # from BrainChart instructions

# other required info
dhcp['study'] = 'dHCP_NewEstimation'  # as dHCP was used for original model fitting, we want to make sure that random effects of study are calculated again
#dhcp['study'] = [f'dHCP_{i}' for i in dhcp['scan']]

dhcp['fs_version'] = 'Custom' 
dhcp['country'] = 'Multisite'
dhcp['run'] = 1
dhcp['session'] = 1

# reshape
all_idps = desikan_idps_bilateral + ctv_columns
dhcp_final = reorder_vars(['participant', 'Age', 'age_days', 'sex', 'study', 'fs_version','country', 'run', 
                            'session', 'dx'], dhcp, all_idps)

# save
print('Final data shape:', dhcp_final.shape)
dhcp_final.to_csv(os.path.join(dhcp_out, f'dHCP_{brain_measure}_preprocessed.csv'), index=False)

Final data shape: (695, 59)


# Stats
Summary stats shown in Supp Table S1.

In [18]:
dhcp_final['age_weeks'] = dhcp_final['age_days'] / 7

dhcp_final_pt = dhcp_final[dhcp_final['dx'] == 'preterm']
dhcp_final_cn = dhcp_final[dhcp_final['dx'] == 'CN']

# summary statistics
dhcp_sample_stats = dict()
for tp in ["term_equivalent"]:
    print(f"---- STATS FOR {tp} SCANS ----")
    cn = dhcp_final.loc[(dhcp_final.scan==tp) & (dhcp_final.dx=='CN')]
    pt = dhcp_final.loc[(dhcp_final.scan==tp) & (dhcp_final.dx=='preterm')]

    print(f"Data available for {len(cn)} term subjects and {len(pt)} preterm subjects")

    # age
    dhcp_sample_stats[("CN", tp, "Age [weeks]")] = cn['age_weeks'].describe()
    dhcp_sample_stats[("preterm", tp, "Age [weeks]")] = pt['age_weeks'].describe()
    
    # sex
    dhcp_sample_stats[("CN", tp, "Male [%]")] = pd.Series({
        "count": len(cn.sex),
        "%": (cn['sex'] == "Male").sum() / len(cn.sex) * 100
    })
    
    dhcp_sample_stats[("preterm", tp, "Male [%]")] = pd.Series({
        "count": len(pt.sex),
        "%": (pt['sex'] == "Male").sum() / len(pt.sex) * 100 
    })
    
    # GA
    dhcp_sample_stats[("CN", tp, "GA [weeks]")] = cn['GA'].describe()
    dhcp_sample_stats[("preterm", tp, "GA [weeks]")] = pt['GA'].describe()
    
    # BW
    dhcp_sample_stats[("CN", tp, "BW [g]")] = cn['BW'].describe()
    dhcp_sample_stats[("preterm", tp, "BW [g]")] = pt['BW'].describe()
    
    # cognition
    dhcp_sample_stats[("CN", tp, "Bayley]")] = cn['bsid_cog_composite'].describe()
    dhcp_sample_stats[("preterm", tp, "Bayley")] = pt['bsid_cog_composite'].describe()
    

dhcp_sample_stats = pd.DataFrame(dhcp_sample_stats).sort_index(axis=1)
dhcp_sample_stats = dhcp_sample_stats.loc[["count", "mean", "%", "std", "50%", "min", "max"],:]
dhcp_sample_stats = dhcp_sample_stats.rename(index={"count":"n", "50%":"median", "std":"sd"})
dhcp_sample_stats = dhcp_sample_stats.round(2)
dhcp_sample_stats

---- STATS FOR term_equivalent SCANS ----
Data available for 564 term subjects and 131 preterm subjects


CN                                              preterm  \
       term_equivalent                                      term_equivalent   
           Age [weeks]   BW [g] Bayley] GA [weeks] Male [%]     Age [weeks]   
n               564.00   451.00  452.00     564.00   564.00          131.00   
mean             41.40  3394.01  101.05      40.00      NaN           40.87   
%                  NaN      NaN     NaN        NaN    53.19             NaN   
sd                1.72   494.57   10.92       1.27      NaN            2.09   
median           41.43  3420.00  100.00      40.14      NaN           40.86   
min              37.43  1820.00   55.00      37.00      NaN           37.00   
max              44.86  4750.00  125.00      43.00      NaN           45.14   

                                             
                                             
         BW [g]  Bayley GA [weeks] Male [%]  
n        101.00  101.00     131.00   131.00  
mean    1714.27   98.91      31.78      NaN  
%           NaN     NaN        NaN    54.96  
sd       720.23   12.78       3.84      NaN  
median  1640.00  100.00      32.14      NaN  
min      540.00   55.00      23.57      NaN  
max     3280.00  125.00      36.86      NaN

In [19]:
# test significance
from scipy.stats import ttest_ind, chi2_contingency
contingency_table = pd.crosstab(dhcp_final['dx'], dhcp_final['sex'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-squared test for sex difference between groups: Chi={chi2:.3f}, p-value: {p:.3f}")

t, p = ttest_ind(dhcp_final_pt['Age'], dhcp_final_cn['Age'], nan_policy='omit', equal_var=True)
print(f"t-test for Age at scan: t={t:.3f}, p-value: {p:.3f}")

t, p = ttest_ind(dhcp_final_pt['bsid_cog_composite'], dhcp_final_cn['bsid_cog_composite'], nan_policy='omit', equal_var=True)
print(f"t-test for Bayley scores: t={t:.3f}, p-value: {p:.3f}")

t, p = ttest_ind(dhcp_final_pt['GA'], dhcp_final_cn['GA'], nan_policy='omit', equal_var=True)
print(f"t-test for GA scores: t={t:.3f}, p-value: {p:.2e}")

t, p = ttest_ind(dhcp_final_pt['BW'], dhcp_final_cn['BW'], nan_policy='omit', equal_var=True)
print(f"t-test for BW scores: t={t:.3f}, p-value: {p:.2e}")

Chi-squared test for sex difference between groups: Chi=0.072, p-value: 0.788
t-test for Age at scan: t=-3.036, p-value: 0.002
t-test for Bayley scores: t=-1.724, p-value: 0.085
t-test for GA scores: t=-42.063, p-value: 5.85e-193
t-test for BW scores: t=-28.121, p-value: 1.67e-108
